In [27]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e2/sample_submission.csv
/kaggle/input/playground-series-s5e2/train.csv
/kaggle/input/playground-series-s5e2/test.csv
/kaggle/input/playground-series-s5e2/training_extra.csv


In [28]:
train = pd.read_csv('/kaggle/input/playground-series-s5e2/train.csv', index_col=['id'])
test = pd.read_csv('/kaggle/input/playground-series-s5e2/test.csv', index_col=['id'])

train

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
id,,,,,,,,,,
0,Jansport,Leather,Medium,7.0,Yes,No,Tote,Black,11.611723,112.15875
1,Jansport,Canvas,Small,10.0,Yes,Yes,Messenger,Green,27.078537,68.88056
2,Under Armour,Leather,Small,2.0,Yes,No,Messenger,Red,16.643760,39.17320
3,Nike,Nylon,Small,8.0,Yes,No,Messenger,Green,12.937220,80.60793
4,Adidas,Canvas,Medium,1.0,Yes,Yes,Messenger,Green,17.749338,86.02312
...,...,...,...,...,...,...,...,...,...,...
299995,Adidas,Leather,Small,9.0,No,No,Tote,Blue,12.730812,129.99749
299996,Jansport,Leather,Large,6.0,No,Yes,Tote,Blue,26.633182,19.85819
299997,Puma,Canvas,Large,9.0,Yes,Yes,Backpack,Pink,11.898250,111.41364


In [29]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 300000 entries, 0 to 299999
Data columns (total 10 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Brand                 290295 non-null  object 
 1   Material              291653 non-null  object 
 2   Size                  293405 non-null  object 
 3   Compartments          300000 non-null  float64
 4   Laptop Compartment    292556 non-null  object 
 5   Waterproof            292950 non-null  object 
 6   Style                 292030 non-null  object 
 7   Color                 290050 non-null  object 
 8   Weight Capacity (kg)  299862 non-null  float64
 9   Price                 300000 non-null  float64
dtypes: float64(3), object(7)
memory usage: 25.2+ MB


*Create a pairplot*

*Get numeric columns and categorical columns*

In [30]:
num_df = train.select_dtypes(include=['number']).drop('Price', axis=1)
cat_df = train.select_dtypes(include=['object'])

num_cols = num_df.columns
cat_cols = cat_df.columns

num_df

,Compartments,Weight Capacity (kg)
id,,
0,7.0,11.611723
1,10.0,27.078537
2,2.0,16.643760
3,8.0,12.937220
4,1.0,17.749338
...,...,...
299995,9.0,12.730812
299996,6.0,26.633182
299997,9.0,11.898250


*Seperate categorical columns into ordinal (to be ordinal encoded) and non-ordinal (to be one-hot encoded)*

# TODO: Try changing the ordinal and non-ordinal columns around

In [31]:
ord_cols = ['Size', 'Laptop Compartment', 'Waterproof']
oh_cols = ['Brand', 'Material', 'Style', 'Color']

*Get unique values*

In [32]:
for col in ord_df.columns:
    print(f'col: {col}\n\
values: {ord_df[col].unique()}')

col: Size
values: ['Medium' 'Small' 'Large' nan]
col: Laptop Compartment
values: ['Yes' 'No' nan]
col: Waterproof
values: ['No' 'Yes' nan]


 *Define ordering for each value*

In [33]:
size_categories = ['Small', 'Medium', 'Large']
yesno_categories = ['No', 'Yes']

*Create transformers for each type of column*

In [50]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler

# numerical_transformer = [
#     ('std_scaler', StandardScaler(), num_cols)
# ]

# ordinal_transformers = [
#     ('size_encoder', OrdinalEncoder(categories=[size_categories]), ['Size']),
#     ('yesno_encoder', OrdinalEncoder(categories=[yesno_categories]), ['Laptop Compartment', 'Waterproof'])
# ]

# num_pipeline = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='median')),
#     ('scale', ColumnTransformer(transformers=numerical_transformer, remainder='Passthrough'))
# ])

# ordinal_pipeline = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='most_frequent')),
#     ('encode', ColumnTransformer(transformers=ordinal_transformers, remainder='Passthrough'))
# ])

num_transformer = Pipeline(
    steps=[
        ('num_impute', SimpleImputer(strategy='median')),
        ('scale', StandardScaler()),
    ]
)

num_df = pd.DataFrame(num_transformer.fit_transform(num_df))
num_df.columns = num_cols


# For ordinal columns

ord_imputer = SimpleImputer(strategy='most_frequent')

ord_df = pd.DataFrame(ord_imputer.fit_transform(ord_df))
ord_df.columns = ord_cols

size_df = ord_df[['Size']]
yesno_df = ord_df[['Laptop Compartment', 'Waterproof']]

size_df = pd.DataFrame(OrdinalEncoder(categories=[size_categories]).fit_transform(size_df))
size_df

ValueError: Shape mismatch: if categories is an array, it has to be of shape (n_features,).

In [39]:
print(ord_df.info())
ord_df

<class 'pandas.core.frame.DataFrame'>
Index: 300000 entries, 0 to 299999
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   Size                293405 non-null  object
 1   Laptop Compartment  292556 non-null  object
 2   Waterproof          292950 non-null  object
dtypes: object(3)
memory usage: 9.2+ MB
None


,Size,Laptop Compartment,Waterproof
id,,,
0,Medium,Yes,No
1,Small,Yes,Yes
2,Small,Yes,No
3,Small,Yes,No
4,Medium,Yes,Yes
...,...,...,...
299995,Small,No,No
299996,Large,No,Yes
299997,Large,Yes,Yes
